#### model 분석하기

In [12]:
import torch
import torch.nn as nn
import torch.nn.init as init
import timm
import sys

class PromptInput(nn.Module):
    def __init__(self, num_prompts, embed_dim = 768, num_layers = 12):
        super().__init__()
        self.num_prompts = num_prompts
        self.embed_dim = embed_dim

        # Initialize prompt embeddings
        self.prompts = nn.Parameter(torch.zeros(num_layers, num_prompts, embed_dim))

        init.kaiming_uniform_(self.prompts)

    def prepend_prompt(self, x, layer_idx):

        batch_size = x.shape[0]

        prompt_tokens = self.prompts[layer_idx,:,:].expand(batch_size,-1,-1)

        if layer_idx == 0:
            x = torch.cat((x[:, :1, :], prompt_tokens, x[:,1:,:]), dim = 1) # => [batch_size, cls_token + prompt_tokens + seq_len, embed_dim]
            
        else:
            x = torch.cat((x[:, :1, :], prompt_tokens, x[:, (1+self.num_prompts):, :]), dim=1) # 이미 추가된 prompt를 제외한 나머지 추가

        return x 
    

# prompt 추가 vit

class Vpt_ViT(nn.Module):
    def __init__(self, pretrained_model= 'vit_base_patch16_224',img_size=32, patch_size=4, num_classes=10, embed_dim=768, depth=12, num_heads=12, mlp_ratio=4., qkv_bias=True, drop_rate=0., attn_drop_rate=0., drop_path_rate=0.):
        super().__init__()
        self.prompt_embedding = PromptInput(num_prompts=100, embed_dim= 768, num_layers= depth)

        #timm을 이용한 pretrained_model 적용
        self.model = timm.create_model(pretrained_model, pretrained = True, img_size = img_size, patch_size = patch_size, num_classes = num_classes)
        self.model.cls_token.requires_grad = False
        self.model.pos_embed.requires_grad = False
        for param in self.model.patch_embed.proj.parameters():
            param.requires_grad = False
        
    def forward(self, x):
        x = self.model.forward_features(x)
        print("1: ",x.shape)
        for idx, block in enumerate(self.model.blocks):
            x = self.prompt_embedding.prepend_prompt(x, idx)
            # print(f"{idx}의 shpae : {x.shape}")
            x = block(x)
        x = self.model.norm(x)  # Final layer normalization
        x = self.model.forward_head(x)
        return x

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Create the model

model = Vpt_ViT(pretrained_model='vit_base_patch16_224', img_size=32, patch_size=4, num_classes=10)
model = model.to(device=device)

# Example input: random tensor with CIFAR-10 dimensions (batch_size, channels, height, width)
input_tensor = torch.randn(8, 3, 32, 32).to(device)  # Batch of 8 images, 3 channels, 32x32 pixels

# Pass the input through the model
output = model(input_tensor)  # Forward pass, should print block indices

for name, param in model.named_parameters():
    if 'blocks' in name :
        param.requires_grad = False
    print(f"Layer: {name} | Requires Grad: {param.requires_grad} | Shape: {param.shape}")


1:  torch.Size([8, 65, 768])
Layer: prompt_embedding.prompts | Requires Grad: True | Shape: torch.Size([12, 100, 768])
Layer: model.cls_token | Requires Grad: False | Shape: torch.Size([1, 1, 768])
Layer: model.pos_embed | Requires Grad: False | Shape: torch.Size([1, 65, 768])
Layer: model.patch_embed.proj.weight | Requires Grad: True | Shape: torch.Size([768, 3, 4, 4])
Layer: model.patch_embed.proj.bias | Requires Grad: True | Shape: torch.Size([768])
Layer: model.blocks.0.norm1.weight | Requires Grad: False | Shape: torch.Size([768])
Layer: model.blocks.0.norm1.bias | Requires Grad: False | Shape: torch.Size([768])
Layer: model.blocks.0.attn.qkv.weight | Requires Grad: False | Shape: torch.Size([2304, 768])
Layer: model.blocks.0.attn.qkv.bias | Requires Grad: False | Shape: torch.Size([2304])
Layer: model.blocks.0.attn.proj.weight | Requires Grad: False | Shape: torch.Size([768, 768])
Layer: model.blocks.0.attn.proj.bias | Requires Grad: False | Shape: torch.Size([768])
Layer: model.

In [3]:
import torch
import torch.nn as nn
import torch.nn.init as init
import timm

class PromptInput(nn.Module):
    def __init__(self, num_prompts, embed_dim=768, num_layers=12):
        super().__init__()
        self.num_prompts = num_prompts
        self.embed_dim = embed_dim

        # Initialize prompt embeddings
        self.prompts = nn.Parameter(torch.zeros(num_layers, num_prompts, embed_dim))
        init.kaiming_uniform_(self.prompts)

    def prepend_prompt(self, x, layer_idx):
        batch_size = x.shape[0]
        prompt_tokens = self.prompts[layer_idx, :, :].expand(batch_size, -1, -1)

        if layer_idx == 0:
            x = torch.cat((x[:, :1, :], prompt_tokens, x[:, 1:, :]), dim=1)  # => [batch_size, cls_token + prompt_tokens + seq_len, embed_dim]
        else:
            x = torch.cat((x[:, :1, :], prompt_tokens, x[:, (1 + self.num_prompts):, :]), dim=1)  # 이미 추가된 prompt를 제외한 나머지 추가

        return x

# VPT with ViT model
class Vpt_ViT(nn.Module):
    def __init__(self, pretrained_model='vit_base_patch16_224', img_size=32, patch_size=4, num_classes=10, embed_dim=768, depth=12):
        super().__init__()
        self.prompt_embedding = PromptInput(num_prompts=100, embed_dim=768, num_layers=depth)

        # Load the pretrained model from timm
        self.model = timm.create_model(pretrained_model, pretrained=True, img_size=img_size, patch_size=patch_size, num_classes=num_classes)
    
    def forward(self, x):
        x = self.model.forward_features(x)
        for idx, block in enumerate(self.model.blocks):
            x = self.prompt_embedding.prepend_prompt(x, idx)
            print(f"{idx}의 shpae : {x.shape}")
            x = block(x)
        x = self.model.norm(x)  # Final layer normalization
        x = self.model.forward_head(x)
        return x

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Create the model
model = Vpt_ViT(pretrained_model='vit_base_patch16_224', img_size=32, patch_size=4, num_classes=10)
model = model.to(device=device)

# Example input: random tensor with CIFAR-10 dimensions (batch_size, channels, height, width)
input_tensor = torch.randn(8, 3, 32, 32).to(device)  # Batch of 8 images, 3 channels, 32x32 pixels

# Pass the input through the model
output = model(input_tensor)  # Forward pass, should print block indices



1:  torch.Size([8, 65, 768])
Layer 0 before block: torch.Size([8, 165, 768])
Layer 0 after block: torch.Size([8, 165, 768])
Layer 1 before block: torch.Size([8, 165, 768])
Layer 1 after block: torch.Size([8, 165, 768])
Layer 2 before block: torch.Size([8, 165, 768])
Layer 2 after block: torch.Size([8, 165, 768])
Layer 3 before block: torch.Size([8, 165, 768])
Layer 3 after block: torch.Size([8, 165, 768])
Layer 4 before block: torch.Size([8, 165, 768])
Layer 4 after block: torch.Size([8, 165, 768])
Layer 5 before block: torch.Size([8, 165, 768])
Layer 5 after block: torch.Size([8, 165, 768])
Layer 6 before block: torch.Size([8, 165, 768])
Layer 6 after block: torch.Size([8, 165, 768])
Layer 7 before block: torch.Size([8, 165, 768])
Layer 7 after block: torch.Size([8, 165, 768])
Layer 8 before block: torch.Size([8, 165, 768])
Layer 8 after block: torch.Size([8, 165, 768])
Layer 9 before block: torch.Size([8, 165, 768])
Layer 9 after block: torch.Size([8, 165, 768])
Layer 10 before block